In [1]:
import gdspy
import numpy

#creates a cell for the TWPA to live in. Required for the gdsII format.
lib = gdspy.GdsLibrary()
cell = lib.new_cell('TWPA')

In [2]:
#padlength is the x-axis extent of the TWPA's ground plane, padwidth is the y-axis extent of the TWPA's ground plane 
padlength = 800
padwidth = padlength

#wirbondpadlength is the x-axis extent of the wire-bond pads at the edge of the TWPA, wirbondpadwidth is the y-axis extent of the wire-bond pads at the edge of the TWPA,
#wirbondpadgap is the spacing between the horizontal edges of the wire-bond pads and the ground plane, 
#wirebondtaperlength is the length of the tapered line between the wire-bond pads and the lead lines
wirebondpadlength = 30
wirebondpadwidth = 15
wirebondpadgap = 6.2
wirebondpadtaperlength = 15

#inputoutputturnradius is the radius if the quarter and half-circle that connects the lead lines to the center spiral
inputoutputturnradius = 10

#radius is the outer radius of the center spiral, radius radio is the ratio between the inner and outer radius of the center spiral (i.e inner radius = radiusratio*radius)
#turns is the number of the turns the center spiral has, spiralcenterangle is the angle from vertical of the center S-shape
radius = 325
radiusratio = 0.1
turns = 20
spiralcenterangle = 0

#segments is the number of seperate pieces in a unit cell of the desired photonic crystal
#For example: a repeating pattern of a short and wide segment and a long and narrow segments would have segments = 2
segments = 6

#lengthfunction is what defines the length of each of the segments(variable) segments and their order, counting up from zero. 
#When fed an integer that corresponds to a segment, the functon returns the length of that segment.
def lengthfunction(n):
    if n == 1:
        y = 410.1
    if n == 3 or n == 5:
        y = 126.6
    else:
        y = 1510
    return y

#widthfunction is what defines the width of each of the segments(variable) segments and their order, counting up from zero. Order of the widths should correspond to the order set in lengthfunction
#When fed an integer that corresponds to a segment and the fractional position along the spiral (i.e. x=0 at the start of spiral, x=1 at the end of the spiral), 
#the functon returns the width of that segment.
def widthfunction(n, x):
    y = 0.2
    return y

#gapfunction is what defines the gap of each of the segments(variable) segments and their order, counting up from zero. 
#Order of the gaps should correspond to the order set in lengthfunction and widthfunction
#When fed an integer that corresponds to a segment and the fractional position along the spiral (i.e. x=0 at the start of spiral, x=1 at the end of the spiral), 
#the functon returns the gap of that segment.
def gapfunction(n, x):
    if (n%2) == 1:
        y = 0.13
    else:
        y = 0.18
    return y

#spirallaterratio is the proportion of the length of a spiral that tapers from the initially defined (x = 0) values to the final (x = 1) defined values
spiraltaperratio = 0.1 #100000/(numpy.pi*(turns+0.5)*0.5*((1 + radiusratio)*radius))

#tol is how closely gdspy approximates a curve with polygons. lower is more accurate but takes much longer to generate
tol = 0.0001

slices = 50

In [3]:
#Calculates the length of a single unit of photonic crystal and stores it as unitlength
unitlength = 0
for n in range(segments):
    unitlength += lengthfunction(n)

#returns the number of the segement within a unit. x must be 0 <= x <= unitlength
def locationinunit(x):
    pos = 0
    location = -1
    for n in range(segments):
        if (pos <= x) and (x < pos + lengthfunction(n)):
            location = n
        pos += lengthfunction(n)
    return location

#returns the end length of the nth segment within a unit
def endlengthofsegment(n):
    endx = 0
    for c in range(n):
        endx += lengthfunction(c)
    return endx

#singlespirallength is the length of a single spiral
singlespirallength = numpy.pi*(turns+0.5)*0.5*((1 + radiusratio)*radius)

#unitsperspiral is the number of unit cells of photonic crystal constained within a single spiral.
unitsperspiral = singlespirallength/unitlength

#unitremainder is the fractional part of unitsperspiral
#unitremainder is the length of the parital unit contained within a single spiral
unitremainder = unitsperspiral - numpy.trunc(unitsperspiral)
unitremainderlength = unitremainder * unitlength

#radius, angle, and derivitives to calculate the next four values
r1 = radius - (1 - radiusratio)*radius
theta1 = (turns+0.5) * numpy.pi
dx_du1 = (1-radiusratio)*radius*numpy.cos(theta1) + r1*numpy.sin(theta1)*(turns+0.5)*numpy.pi
dy_du1 = -(1-radiusratio)*radius*numpy.sin(theta1) + r1*numpy.cos(theta1)*(turns+0.5)*numpy.pi

#values calculated to make sure the center portion of the TWPA connects and aligns correctly.
#dy_dx1 is the derivitive of the center of the spirals' ends
#spiralendangle is the angle that the ends of the spiral make relative to the veritcal plane
#centerradius is the radius that the center turns need to have in order to conenct with the center segments in order for the segments to connect with the center ends of the spirals
#centerstarightength is the length that the center segments need to have in order to conenct with the center turns and the center ends of the spirals
dy_dx1 = dy_du1/dx_du1
spiralendangle = numpy.arctan(dy_dx1)
centerturnradius = (radiusratio*radius)/(1 + numpy.cos(spiralendangle) + numpy.sin(spiralendangle)*dy_dx1)

#leadlength is the length required to connect the wire-bond pads' taper to the turns connecting to the center spiral
leadlength = padlength/2 - wirebondpadlength - wirebondpadtaperlength - radius - 3*inputoutputturnradius - (((1 - radiusratio)*radius*spiralcenterangle)/((turns + 0.5)*numpy.pi))

#totallength is the center length of the TWPA's center line from the end of one spiral to the end of the other
#totals full units is the number of full photonic crystal unit cells withing the center spiral portion of the TWPA
totallength = 2*(singlespirallength + centerturnradius*dy_dx1 + 2*centerturnradius*numpy.pi)
totalfullunits = numpy.trunc(totallength/unitlength)

#thinwirelength is the length of the center spiral which is at its final width between tapers.
thinwirelength = 2*((1 - spiraltaperratio)*singlespirallength + centerturnradius*dy_dx1 + 2*centerturnradius*numpy.pi)

#spacing is the space between centers of the turns of the center spiral
spacing = ((1 - radiusratio)*radius)/(turns + 0.5)

#a function that takes two arguments and always returns 0. Added to make the gapfunc 0 when making the thin center line and not the ground plane with the thicker center line cutout.
def zerofunction(a, b):
    return 0

In [4]:
#creates the center line of a TWPA that has width widthfunc + 2*gapfunc. If makegroundplane=1 centerline will subtract the centerline from a rectangle to serve as a ground plane and add it to the cell, 
#if makegroundplane=0 centerline will add the center line to the cell.
def centerline(widthfunc, gapfunc, makegroundplane): 
    
    #creates the TWPAinputline path object with an initial width of wirebondpadwidth + 2*wirebondpadgap*makegroundplane and a start position of (0, 0)
    TWPAinputline = gdspy.Path(wirebondpadwidth + 2*wirebondpadgap*makegroundplane, (0, 0))
    
    #adds a straight segment of length wirebondpathlangth in the +x direction to the end of the path. Since a new width was not specified the segment will use the path's initial width
    TWPAinputline.segment(wirebondpadlength, '+x')

    #adds a straight segment of length wirebondtaperlength to the end of the TWPAinputline path. 
    #Since a new direction was not specified the segment will be oriented in the same direction as the end of the previous part of the path.
    #Since a final_width was defined, the segment will linearly taper from the final_width of the previous part of the path to the defined final_width of widthfunc(0, 0) + 2*gapfunc(0, 0)
    TWPAinputline.segment(wirebondpadtaperlength, final_width = widthfunc(0, 0) + 2*gapfunc(0, 0))

    #adds a straight segment of length (leadlength) to the end of the TWPAinputline path. This length was chosen to fit the lead line between the turns leading into the centerspiral and the wire-bond taper segment
    TWPAinputline.segment(leadlength)

    #adds a turn with radius inputoutputturnradius to the end of the TWPAinputline path. Since 'r' is specified as the angle of the turn, it will be a quarter-circle right turn
    TWPAinputline.turn(inputoutputturnradius, 'r', tolerance = tol)

    #adds a turn with radius inputoutputturnradius to the end of the TWPAinputline path. Since 'll' is specified as the angle of the turn, it will be a half-circle left turn
    TWPAinputline.turn(inputoutputturnradius, 'll', tolerance = tol)
    
    #adds a straight segment of length inputoutputturnradius to the end of the TWPAinputline path. This segment is added to keep both wire-bond pads' center and the center spiral's center centered on the x-axis
    TWPAinputline.segment(inputoutputturnradius)
    
    #If a spiralcenterangle is nonzero, adds a 'back continuous' portion of the spiral to the end of the TWPAinputline path. This spiral portion is added to bridge the gap between the input line
    #and the rotated center spiral.
    if spiralcenterangle != 0:
        #defines the spiral 'back continuous' portion's shape.
        def spiralinput(u):
            r = radius + (((1 - radiusratio)*radius*spiralcenterangle)/((turns + 0.5)*numpy.pi))*(1 - u)
            theta = u*spiralcenterangle
            x = -r * numpy.cos(theta) + radius + (((1 - radiusratio)*radius*spiralcenterangle)/((turns + 0.5)*numpy.pi))
            y = r * numpy.sin(theta)
            return (x, y)

        #defines the spiral 'back continuous' portion's derivitive.
        def dspiralinput_du(u):
            r = radius + (((1 - radiusratio)*radius*spiralcenterangle)/((turns + 0.5)*numpy.pi))*(1 - u)
            theta = u*spiralcenterangle
            if 0 < u < 1:
                dx = -(((1 - radiusratio)*radius*spiralcenterangle)/((turns + 0.5)*numpy.pi))*numpy.cos(theta) + r*numpy.sin(theta)*spiralcenterangle
                dy = (((1 - radiusratio)*radius*spiralcenterangle)/((turns + 0.5)*numpy.pi))*numpy.sin(theta) + r*numpy.cos(theta)*spiralcenterangle
            else:
                dx = numpy.sin(theta)
                dy = numpy.cos(theta)
            return (dx, dy)

        #adds the the spiral 'back continuous' portion to the end of the TWPAinputline path.
        TWPAinputline.parametric(spiralinput, dspiralinput_du, tolerance = tol, max_points = 8190, relative = True)
    
    #creates the TWPAspiralline path object with an initial width of widthfunc(0, 0) + 2*gapfunc(0, 0) and a start position of (-radius, 0) which places the center of the spiral at (0, 0)
    TWPAspiralline = gdspy.Path(widthfunc(0, 0) + 2*gapfunc(0, 0), (-radius, 0))
    
    #spiral2 is the function that defines the spiral's shape. It is a parametric function of u, a variable which varies from 0 to 1. 
    #First the polar equation is defined, then it is translated into cartesian coordinates which are returned by the function
    def spiral1(u):
        r = radius - (1 - radiusratio)*radius*u
        theta = (turns+0.5) * u * numpy.pi
        x = -r * numpy.cos(theta) + radius
        y = r * numpy.sin(theta)
        return (x, y)

    #dspiral1_du is the function which defines the siral's derivitive. The path's Parametric method uses the derivitive to calculated the direction of the vector of the parametric portion of the path.
    #to simplify the start of the spiral a condition has been added which forces dspiral1_du(0) to be (0, 1) so that the start face of the spiral has a normal vector point in the +y direction, 
    #which results in the start face being horizontal, therefore fully connecting to the input portion of the design. If a derivitive function is not defined, the parametric method will calculate the
    #derivitive numerically from the defined function.
    def dspiral1_du(u):
        r = radius - (1 - radiusratio)*radius*u
        theta = (turns+0.5) * u * numpy.pi
        if u == 0:
            dx = 0
            dy = 1
        else:
            dx = (1 - radiusratio)*radius*numpy.cos((turns+0.5)*numpy.pi*u) + r*numpy.sin((turns+0.5)*numpy.pi*u)*(turns+0.5)*numpy.pi
            dy = -(1 - radiusratio)*radius*numpy.sin((turns+0.5)*numpy.pi*u) + r*numpy.cos((turns+0.5)*numpy.pi*u)*(turns+0.5)*numpy.pi
        return (dx, dy)

    #widfuncspiral1 is the function which defines the spiral's width at a specific value of u. lengthalongspiral is the length of spiral created at a given value of u,
    #and lengthalong unit is how far along a unit cell a given value of u is. If the fraction lengthalongspiral/singlespirallength is less than spiraltaperratio the x value of 
    #(lengthalongspiral/singlespirallength)/spiraltaperratio) is given to widthfunction to taper as defined, if not, the x value of 1 (end of taper) is fed instead. locationinunit(lengthalongunit) gives
    #the n value of the correct segment for a given value of u. This function the returns the correct spiral width for a given value of u.
    def widfuncspiral1(u):
        lengthalongspiral = numpy.pi*(turns+0.5)*0.5*radius*u*(2-(1 - radiusratio)*u)
        lengthalongunit = (lengthalongspiral)%unitlength
        if (lengthalongspiral/singlespirallength) < spiraltaperratio:
            width = widthfunc(locationinunit(lengthalongunit), ((lengthalongspiral/singlespirallength)/spiraltaperratio)) + 2*gapfunc(locationinunit(lengthalongunit), ((lengthalongspiral/singlespirallength)/spiraltaperratio))
        else:
            width = widthfunc(locationinunit(lengthalongunit), 1) + 2*gapfunc(locationinunit(lengthalongunit), 1)
        return width

    #uses the three previously defined functions to add the desired spiral to the end of the TWPAspiralline path. Each of the functions is fed a value of u, which varies from 0 to 1, to sweep out the shape.
    #Since gdsII files only contain polygons and a spiral is a curve which much be approximated by polygons, tolerance is specified as well as the maximum number of vertices a polygon is allowed
    #to have. relaive = True allows the paametric shape to be moved so that it starts at the previous end of the path, but only if u=0 results in each of the defined functions returning (0, 0).
    TWPAspiralline.parametric(spiral1, dspiral1_du, tolerance = tol, max_points = 8190, final_width = widfuncspiral1, relative = True)
    
    #This function defines a straight line in u-parametric polar coordinates, which then is transformed into cartesion coordinates which are then returned.
    def segment1(u):
        if  (turns%2) == 1:
            r = u*centerturnradius*dy_dx1
        else:
            r = -u*centerturnradius*dy_dx1
        x = r*numpy.cos(spiralendangle)
        y = r*numpy.sin(spiralendangle)
        return (x,y)

    #Defines the width of the parametric segment for varous values of u. 
    def widfuncsegment1(u):
        lengthalongspiral = -centerturnradius*dy_dx1*u
        lengthalongunit = (lengthalongspiral + unitremainderlength)%unitlength
        width = widthfunc(locationinunit(lengthalongunit), 1) + 2*gapfunc(locationinunit(lengthalongunit), 1)
        return width

    #adds a parametric line defined above to the end of the TWPAspiralline path. This segment is parametric due to normal path.segment objects only being able to linearly taper their widths instead of 
    #continuously varying them.
    TWPAspiralline.parametric(segment1, final_width = widfuncsegment1, max_points = 8190, tolerance = tol, relative = True)
    
    #defines a parametric turn with a start angle of spiralendangle + numpy.pi*(turns%2) and an end angle of (numpy.pi + spiralendangle) + spiralendangle + numpy.pi*(turns%2) and constant radius
    #of centerturnradius. The start angle was chosen to keep the turn and previous segment flush due to the archimedian spiral's end face not being vertially aligned.
    def turn1(u):
        theta = -u * (numpy.pi + spiralendangle) + spiralendangle + numpy.pi*(turns%2)
        r = centerturnradius
        x = -r * numpy.sin(theta) + r * numpy.sin(spiralendangle + numpy.pi*(turns%2))
        y = r * numpy.cos(theta)  - r * numpy.cos(spiralendangle + numpy.pi*(turns%2))
        return (x, y)
    
    #defines the first turn's derivitive
    def dturn1_du(u):
        theta = -u * (numpy.pi + spiralendangle) + spiralendangle + numpy.pi*(turns%2)
        r = centerturnradius
        if u > 0:
            dx = -r*(theta/u)*numpy.cos(theta)
            dy = -r*(theta/u)*numpy.sin(theta)
        else:
            dx = (1 - radiusratio)*radius*numpy.cos((turns+0.5)*numpy.pi) + radius*radiusratio*numpy.sin((turns+0.5)*numpy.pi)*(turns+0.5)*numpy.pi
            dy = -(1 - radiusratio)*radius*numpy.sin((turns+0.5)*numpy.pi) + radius*radiusratio*numpy.cos((turns+0.5)*numpy.pi)*(turns+0.5)*numpy.pi
        return (dx, dy)
    
    #Defines the width of the parametric turn for varous values of u.
    def widfuncturn1(u):
        lengthalongspiral = centerturnradius*(numpy.pi + spiralendangle)*u
        lengthalongunit = (lengthalongspiral + unitremainderlength - centerturnradius*dy_dx1)%unitlength
        width = widthfunc(locationinunit(lengthalongunit), 1) + 2*gapfunc(locationinunit(lengthalongunit), 1)
        return width
    
    #Adds the parametric turn defined above to the end of the TWPAspiralline path. This turn is parametric due to normal path.turn objects only being able to linearly taper their widths instead of 
    #continuously varying them.
    TWPAspiralline.parametric(turn1, dturn1_du, tolerance = tol, max_points = 8190, final_width = widfuncturn1, relative = True)
    
    #defines a parametric turn with a start angle of numpy.pi*(turns%2) and an end angle of (numpy.pi + spiralendangle) + numpy.pi*(turns%2) and constant radius
    #of centerturnradius. The end angle was chosen to keep the next segment and next spiral flush due to the archimedian spiral's end face not being vertially aligned.
    def turn2(u):
        r = centerturnradius
        theta = u * (numpy.pi + spiralendangle) + numpy.pi*(turns%2)
        x = -r * numpy.sin(theta) + r*numpy.sin(numpy.pi*(turns%2))
        y = r * numpy.cos(theta) - r*numpy.cos(numpy.pi*(turns%2))
        return (x, y)
    
    #defines the second turn's derivitive
    def dturn2_du(u):
        theta = u * (numpy.pi + spiralendangle)
        r = centerturnradius
        if u < 1:
            dx = -r*(numpy.pi + spiralendangle)*numpy.cos(theta)
            dy = -r*(numpy.pi + spiralendangle)*numpy.sin(theta)
        else:
            dx = radiusratio*radius*(turns+0.5)*numpy.pi
            dy = -(1 - radiusratio)*radius
        return (dx, dy)
    
    #Defines the width of the parametric turn for varous values of u.
    def widfuncturn2(u):
        lengthalongspiral = centerturnradius*(numpy.pi + spiralendangle)*u
        lengthalongunit = (lengthalongspiral + unitremainderlength - centerturnradius*dy_dx1 + centerturnradius*(numpy.pi + spiralendangle))%unitlength
        width = widthfunc(locationinunit(lengthalongunit), 1) + 2*gapfunc(locationinunit(lengthalongunit), 1)
        return width
    
    #Adds a parametric turn defined above to the end of the TWPAspiralline path.
    TWPAspiralline.parametric(turn2, dturn2_du, tolerance = tol, max_points = 8190, final_width = widfuncturn2)
    
    #Defines a straight line in u-parametric polar coordinates, which then is transformed into cartesion coordinates which are then returned.
    def segment2(u):
        if  (turns%2) == 1:
            r = u*centerturnradius*dy_dx1
        else:
            r = -u*centerturnradius*dy_dx1
        x = r*numpy.cos(spiralendangle)
        y = r*numpy.sin(spiralendangle)

        return (x,y)
    #Defines the width of the parametric segment for varous values of u.
    def widfuncsegment2(u):
        lengthalongspiral = -centerturnradius*dy_dx1*u
        lengthalongunit = (lengthalongspiral + unitremainderlength - centerturnradius*dy_dx1 + 2*centerturnradius*(numpy.pi + spiralendangle))%unitlength
        width = widthfunc(locationinunit(lengthalongunit), 1) + 2*gapfunc(locationinunit(lengthalongunit), 1)
        return width
    
    #adds the parametric line defined above to the end of the TWPAspiralline path.
    TWPAspiralline.parametric(segment2, final_width = widfuncsegment2, tolerance = tol, max_points = 8190, relative = True)
    
    #defines the second spiral's shape. This spiral starts on its inner radius and spirals outwads to its outer radius.
    def spiral2(u):
        r = radiusratio*radius + (1 - radiusratio)*radius*u
        theta = (turns+0.5) * u * numpy.pi + numpy.pi*(turns%2)
        x = r * numpy.sin(theta) - r*numpy.sin(numpy.pi*(turns%2))
        y = -r * numpy.cos(theta) - radiusratio*radius*(turns%2) + radiusratio*radius*((turns+1)%2)
        return (x, y)

    #defines the derivitive of the second spiral. There is a condition that the end of the spiral's normal vector is pointed in the +y direction.
    def dspiral2_du(u):
        r = radiusratio*radius + (1 - radiusratio)*radius*u
        theta = (turns+0.5) * u * numpy.pi
        if u == 1:
            dx = 0
            dy = 1
        else:
            if (turns%2) == 0:
                dx = ((1 - radiusratio)*radius*numpy.sin((turns+0.5)*numpy.pi*u) + r*numpy.cos((turns+0.5)*numpy.pi*u)*(turns+0.5)*numpy.pi)
                dy = (-(1 - radiusratio)*radius*numpy.cos((turns+0.5)*numpy.pi*u) + r*numpy.sin((turns+0.5)*numpy.pi*u)*(turns+0.5)*numpy.pi)
            else:
                dx = -((1 - radiusratio)*radius*numpy.sin((turns+0.5)*numpy.pi*u) + r*numpy.cos((turns+0.5)*numpy.pi*u)*(turns+0.5)*numpy.pi)
                dy = -(-(1 - radiusratio)*radius*numpy.cos((turns+0.5)*numpy.pi*u) + r*numpy.sin((turns+0.5)*numpy.pi*u)*(turns+0.5)*numpy.pi)
        return (dx, dy)

    #Defines the width as a function of u of the second spiral. Has logic to have the taper be at the end of the spiral instead of teh beginning, as well as conditions to make sure that 
    #if the current feature would end past the end of the spiral, the spiral instead has a width of 
    #widthfunc(0, ((1 - lengthalongspiral/singlespirallength)/spiraltaperratio)) + 2*gapfunc(0, ((1 - lengthalongspiral/singlespirallength)/spiraltaperratio))
    def widfuncspiral2(u):
        lengthalongspiral = numpy.pi*(turns + 0.5)*0.5*radius*u*(2*radiusratio + (1 - radiusratio)*u)
        lengthalongunit = (lengthalongspiral + unitremainderlength + 2*centerturnradius*dy_dx1 + 2*centerturnradius*(numpy.pi + spiralendangle))%unitlength
        lastunitstart = singlespirallength - (unitlength*((totallength/unitlength)%1))
        if (1 - lengthalongspiral/singlespirallength) < spiraltaperratio:
            if (lengthalongspiral > lastunitstart) and (lengthalongspiral + endlengthofsegment(locationinunit(lengthalongunit)) > singlespirallength):
                width = widthfunc(0, ((1 - lengthalongspiral/singlespirallength)/spiraltaperratio)) + 2*gapfunc(0, ((1 - lengthalongspiral/singlespirallength)/spiraltaperratio))
            else:
                width = widthfunc(locationinunit(lengthalongunit), ((1 - lengthalongspiral/singlespirallength)/spiraltaperratio)) + 2*gapfunc(locationinunit(lengthalongunit), ((1 - lengthalongspiral/singlespirallength)/spiraltaperratio))
        else:
            width = widthfunc(locationinunit(lengthalongunit), 1) + 2*gapfunc(locationinunit(lengthalongunit), 1)
        return width
    
    #adds the second spiral defined above to the end of the TWPAspiralline path. 
    TWPAspiralline.parametric(spiral2, dspiral2_du, tolerance = tol, max_points = 8190, final_width = widfuncspiral2, relative = True)
    
    #Rotates the TWPAspiralline path -spiralcenterangle radians around its center (0, 0) then moves it to the center of the pad.
    TWPAspiralline.rotate(-spiralcenterangle)
    TWPAspiralline.translate(padlength/2, 0)
    
    #creates the TWPAoutputline path object as a copy of TWPAinputline
    TWPAoutputline = gdspy.copy(TWPAinputline)
    
    #Rotates the TWPAspiralline path pi radians around its start (0, 0) then moves it to the right edge of the pad.
    TWPAoutputline.rotate(numpy.pi)
    TWPAoutputline.translate(padlength, 0)
    
    #decide whether to add a rectange and subtract the TWPAinputline, TWPAspiralline and TWPAoutputline paths from it, or to add the TWPAinputline, TWPAspiralline and TWPAoutputline paths to the cell.
    if makegroundplane:
        
        groundrectangle = gdspy.Rectangle((0, padwidth/2), (padlength, -padwidth/2))
        
        positions = []
        
        for i in range(1, slices):
            positions.append((padlength/slices)*i)
        
        groundslice = gdspy.slice(groundrectangle, positions, 0, precision = tol)
        
        for i in range(slices):
            groundslice[i] = gdspy.boolean(groundslice[i], TWPAinputline, 'not', precision = tol, max_points = 8190)
        
            groundslice[i] = gdspy.boolean(groundslice[i], TWPAspiralline, 'not', precision = tol, max_points = 8190)

            groundslice[i] = gdspy.boolean(groundslice[i], TWPAoutputline, 'not', precision = tol, max_points = 8190)

            cell.add(groundslice[i])
        
    else:
        cell.add(TWPAinputline)
        
        cell.add(TWPAspiralline)
        
        cell.add(TWPAoutputline)
    
    return

In [5]:
#Creates the center line and the ground plane of a TWPA.
def TWPA():
    centerline(widthfunction, zerofunction, 0)
    centerline(widthfunction, gapfunction, 1)
    return
TWPA()

In [6]:
#Prints some useful information about the generated TWPA
print('The length of the thin-wired TWPA line is approximately ' + str(numpy.trunc(thinwirelength)) + ' um or ' + str(round(thinwirelength/10000, 1)) + ' cm.')
print('There are ' + str(totalfullunits) + ' full photonic crystal units in the TWPA. Each photonic crystal unit has a length of ' + str(unitlength) + ' um.')
print('The spacing between the spirals turns is ' + str(spacing) + ' um.')
print('The lead length is ' + str(leadlength) + ' um.')

The length of the thin-wired TWPA line is approximately 20920.0 um or 2.1 cm.
There are 3.0 full photonic crystal units in the TWPA. Each photonic crystal unit has a length of 6293.200000000001 um.
The spacing between the spirals turns is 14.268292682926829 um.
The lead length is 0.0 um.


In [7]:
#opens the generated TWPA in gdspy's included viewer
gdspy.LayoutViewer(lib)

<gdspy.viewer.LayoutViewer object .!layoutviewer>

In [8]:
#saves the generated TWPA to a file named whatever is in the quotes
lib.write_gds('Test TWPA.gds')